# Dropout



In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

from keras.callbacks import History 

from keras import optimizers

from time import time

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Carichiamo il dataset

In questo notebook addestreremo una rete neurale in grado di comprendere se una recensione relativa a un film è positiva o negativa. A questo scopo utilizzeremo [l'IMDB Moview Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) un dataset di 50.000 recensioni etichettate come positive o negative.
<br>
Possiamo caricare il dataset utilizzando direttamente Keras, il parametro num_words ci serve per impostare il numero massimo di parole più frequenti da selezionare e di conseguenza il numero di features del nostro modello.

In [2]:
from keras.datasets import imdb 

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


Adesso abbiamo due liste, entrambe contenenti 25.000 osservazioni, per addestramento e test della nostra rete. Le recensioni sono già codificate in delle liste di id, un id identifica la posizione della relativa parola all'interno della lista ordinata delle parole che compaiono più spesso all'interno del nostro corpus di testo, con un offset di 3 posizioni. L'offset è utilizzato perché le prime 3 posizioni sono utilizzate per **boh**.

In [3]:
print("La prima recensione del train set contiene %d parole" % len(X_train[0]))
print("IDs delle prime 10 parole delal recensione: %s" % X_train[0][:10])

La prima recensione del train set contiene 218 parole
IDs delle prime 10 parole delal recensione: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]


Ad esempio, la seconda parola della prima recensione ha id 14, quindi corrisponde all'11 parola più frequente all'interno del corpus. Possiamo ottenere la posizione di una parola all'interno della lista delle parole più frequenti tramite il dizionario word_index.

In [4]:
word_index = imdb.get_word_index()
word_index['love']-3

113

La parola 'love' è la 113esima parola più frequente all'interno del nostro corpus (nota l'offset di 3). Per poter ricostruire una frase partendo dagli IDs dobbiamo conoscere la relazione inversa, cioè a quale parola è associato un determinato id.

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

reverse_word_index.get(113+3)

'love'

Adesso possiamo utilizzare il dizionario con la relazione parola->id per ricostruire una frase, facciamolo con la prima frase all'interno del set di addestramento.

In [6]:
decoded_review = [reverse_word_index.get(i - 3, '?') for i in X_train[0]]
decoded_review = ' '.join(decoded_review)

print("REVIEW: "+decoded_review)
print("\n")
print("SENTIMENT [1=Positive/0=Negative]: %d" % y_train[0])

REVIEW: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly ? was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little ? that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big ? for the whole film but these children are amazing and should be ? for what they have done don't you think the whole s

## Preprocessing

Le frasi nel corpus di testo sono già codificate in numeri, ma hanno una lunghezza differente,  per poterle utilizzare per addestrare la nostra rete neurale abbiamo bisogno di codificarle per ottenere un numero di features consistente. Abbiamo diverse tecniche per farlo, utilizziamo la più semplice: il one hot encoding.

In [7]:
def onehot_encoding(data, size):
    onehot = np.zeros((len(data), size))
    for i, d in enumerate(data):
        onehot[i,d] = 1.
    return onehot

La funzione definita prende in input un corpus di testo e per ogni frase crea un numero di variabili di comodo pari al numero di parole totale del corpus.

In [8]:
X_train = onehot_encoding(X_train, 5000)
X_test = onehot_encoding(X_test, 5000)

X_train.shape

(25000, 5000)

Potremmo ottenere risultati migliori utilizzando tecniche più complesse ottimizzate per il natural language processing (leggasi **words embedding**) ma per il momento non complichiamoci troppo la vita e passiamo alla creazione del nostro modello.

## Creiamo la rete neurale

Creiamo un modello di rete neurale profonda con ben 5 strati, 1 di input, 3 nascosti e 1 di output. Trattandosi di un problema di classificazione binaria (recension positiva/negativa) come funzione di attivazione dell'ultimo strato dobbiamo utilizzare la funzione e come funzine di costo la binary crossentropy.

In [9]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(5000,)))
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

l nostro set di addestramento è rappresentato da una matrice sparsa, dato che ogni esempio ha 5000 colonne ma soltanto qualche dozzina di parole, quindi proviamo ad utilizzare la funzione di ottimizzazione **adamax**, che in questi casi dovrebbe portare a risultati migliori dell'adam.

In [10]:
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

Avviamo l'addestramento per soltanto 10 epoche, fidati basteranno.

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=512)

Epoch 1/10
25000/25000 [==============================] - 4s 147us/step - loss: 0.3701 - acc: 0.8403
Epoch 2/10
25000/25000 [==============================] - 3s 130us/step - loss: 0.2001 - acc: 0.9250
Epoch 3/10
25000/25000 [==============================] - 3s 116us/step - loss: 0.1294 - acc: 0.9555
Epoch 4/10
25000/25000 [==============================] - 3s 116us/step - loss: 0.0598 - acc: 0.9858
Epoch 5/10
25000/25000 [==============================] - 3s 122us/step - loss: 0.0210 - acc: 0.9963
Epoch 6/10
25000/25000 [==============================] - 3s 121us/step - loss: 0.0070 - acc: 0.9988
Epoch 7/10
25000/25000 [==============================] - 3s 123us/step - loss: 0.0024 - acc: 0.9998
Epoch 8/10
25000/25000 [==============================] - 3s 125us/step - loss: 0.0011 - acc: 0.9999
Epoch 9/10
25000/25000 [==============================] - 3s 127us/step - loss: 5.6465e-04 - acc: 1.0000
Epoch 10/10
25000/25000 [==============================] - 3s 125us/step - loss: 3.9564

Come vedi dopo pochissimo la nostra rete ha ottenuto risultati sul set di addestramento che tendono alla perfezione, questo ci fa pensare a un problema di overfitting. Verifichiamolo valutando il modello sul set di test.

In [12]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 2s 99us/step


[0.699159887341708, 0.87224]

Qui le metriche sono decisamente più scarse, si tratta di overfitting.

## Applicare il dropout
Cerchiamo di contrastare l'overfitting utilizzando una combinazione di regolarizzazione L2 e Dropout. Possiamo utilizzare il Dropout con Keras aggiungendo un'instanza della classe Dropout come fosse un nuovo strato, questa classe necessità di un unico parametro che raprresenta la percentuale di nodi da mantenere attivi.

In [13]:
from keras.regularizers import l2
from keras.layers import Dropout


model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(5000,), kernel_regularizer=l2(0.1)))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu',kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(8,activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=512)

Epoch 1/100
25000/25000 [==============================] - 5s 207us/step - loss: 12.4596 - acc: 0.5471
Epoch 2/100
25000/25000 [==============================] - 5s 191us/step - loss: 0.9065 - acc: 0.7119
Epoch 3/100
25000/25000 [==============================] - 5s 182us/step - loss: 0.7415 - acc: 0.7999
Epoch 4/100
25000/25000 [==============================] - 4s 173us/step - loss: 0.7077 - acc: 0.8244
Epoch 5/100
25000/25000 [==============================] - 4s 169us/step - loss: 0.6853 - acc: 0.8330
Epoch 6/100
25000/25000 [==============================] - 5s 181us/step - loss: 0.6665 - acc: 0.8405
Epoch 7/100
25000/25000 [==============================] - 4s 153us/step - loss: 0.6505 - acc: 0.8428
Epoch 8/100
25000/25000 [==============================] - 4s 177us/step - loss: 0.6144 - acc: 0.8459
Epoch 9/100
25000/25000 [==============================] - 5s 194us/step - loss: 0.6030 - acc: 0.8500
Epoch 10/100
25000/25000 [==============================] - 5s 189us/step - loss:

25000/25000 [==============================] - 4s 171us/step - loss: 0.4736 - acc: 0.8786
Epoch 81/100
25000/25000 [==============================] - 5s 180us/step - loss: 0.4686 - acc: 0.8773
Epoch 82/100
25000/25000 [==============================] - 4s 172us/step - loss: 0.4715 - acc: 0.8796
Epoch 83/100
25000/25000 [==============================] - 4s 161us/step - loss: 0.4715 - acc: 0.8773
Epoch 84/100
25000/25000 [==============================] - 4s 159us/step - loss: 0.4690 - acc: 0.8823
Epoch 85/100
25000/25000 [==============================] - 4s 153us/step - loss: 0.4725 - acc: 0.8782
Epoch 86/100
25000/25000 [==============================] - 4s 170us/step - loss: 0.4753 - acc: 0.8786
Epoch 87/100
25000/25000 [==============================] - 4s 167us/step - loss: 0.4680 - acc: 0.8807
Epoch 88/100
25000/25000 [==============================] - 4s 169us/step - loss: 0.4694 - acc: 0.8791
Epoch 89/100
25000/25000 [==============================] - 4s 172us/step - loss: 0.46

I risultati dell'addestramento sono più modesti rispetto a prima, ma stavolta non sembrano mostrare segni di overfitting. Verifichiamolo sul set di test

In [14]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 4s 153us/step


[0.4616530714035034, 0.8756]

Questa volta la nostra rete è riuscita a generalizzare su dati sconosciuti il maniera eccellente.

 ## Mettiamo la rete neurale all'opera

Adesso che abbiamo addestrato e validato la nostra rete, perché non metterla in azione ? Ho scritto questa funzione per prendere una recensione e processarla in modo tale da poterla dare in pasto alla nostra rete.

In [15]:
from re import sub

def preprocess(review):
    
    # Rimuoviamo un'eventuale punteggiatura
    review = sub(r'[^\w\s]','',review) 
    # Convertiamo tutto in minuscolo
    review = review.lower()
    # Creiamo un array di parole
    review = review.split(" ")

    # Qui dentro metteremo gli IDs
    review_array = []

    # Iteriamo lungo le parole della recensione
    for word in review:
        # proseguiamo se la parola si trova all'interno
        # della lista di parole del corpus di addestramento
        if word in word_index:
            # estraiamo l'indice della parola 
            index = word_index[word] 
            # Proseguiamo se l'indice è minore o uguale a 5000
            # cioè il numero di parole che abbiamo utilizzato
            # per l'addestramento
            if index <= 5000:
                # aggiungiamo l'indice all'array
                review_array.append(word_index[word]+3)
                
    # Eseguiamo il one hot encoding sulla lista di indici
    review_array = onehot_encoding([review_array],5000)
    
    return review_array

Un valore maggiore dell'output della rete corrisponde ad una recensione maggiormente positiva, scriviamo una semplice funzione per interpretare l'output della rete come il sentiment della recensione.

In [16]:
def prob_to_sentiment(y):
    
    if(prob>0.9): return "fantastica"
    elif(prob>0.75): return "ottima"
    elif(prob>0.55): return "buona" 
    elif(prob>0.45): return "neutrale"
    elif(prob>0.25): return "negativa"
    elif(prob>0.1): return "brutta"
    else: return "pessima"

Testiamo la nostra rete su una recensione che ho preso da internet, relativa a uno dei film più brutti che ho avuto la sciagura di vedere: Paranormal Activity 4.

In [19]:
review = "what a waste of time and cash.. the movie was pointless. with no flow. no questions answered. just a waste. I never review movies but had to share how bad this was..compared to part 1- 2- and 3.... i don't know what else to say other than how misleading the commercial is.. the commercial was cut and spliced with video and audio that didn't even match what happened in the movie... you have been warned. when the movie was over.. people actually Boo'd. hopefully people will spread the word, and save others from throwing their money away. i know die-hard fans will go and give it a shot, but will be disappointed as well. Sinister was better and actually made you jump quite a few times."
x = preprocess(review)
y = model.predict(x)[0]
print("REVIEW: %s" % review)
print("\n")
print("La recesione è %s [%.6f]" % (prob_to_sentiment(y), y))

REVIEW: what a waste of time and cash.. the movie was pointless. with no flow. no questions answered. just a waste. I never review movies but had to share how bad this was..compared to part 1- 2- and 3.... i don't know what else to say other than how misleading the commercial is.. the commercial was cut and spliced with video and audio that didn't even match what happened in the movie... you have been warned. when the movie was over.. people actually Boo'd. hopefully people will spread the word, and save others from throwing their money away. i know die-hard fans will go and give it a shot, but will be disappointed as well. Sinister was better and actually made you jump quite a few times.


La recesione è pessima [0.000689]


La nostra rete indica che la recensione è (ovviamente) pessima, ma proprio tanto tanto. Proviamo adesso con una recensione che riguarda Avengers: Infinity War.

In [21]:
review = "This movie will blow your mind and break your heart - and make you desperate to go back for more. Brave, brilliant and better than it has any right to be."
x = preprocess(review)
y = model.predict(x)

print("REVIEW: %s" % review)
print("\n")
print("La recesione è %s [%.6f]" % (prob_to_sentiment(y), y))

REVIEW: This movie will blow your mind and break your heart - and make you desperate to go back for more. Brave, brilliant and better than it has any right to be.


La recesione è ottima [0.880576]


La rete dice che la recensione è positiva (dai, a chi non è piaciuto questo film ?). Se vuoi divertirti un po' prova a scrivere la tua recensione, deve essere in linua inglese e ovviamente la nostra rete non riesce a comprendere il sarcasmo o frasi ambigue (es. questo film è una bellissima cagata).

In [ ]:
review = input("Write your review: ")
x = preprocess(review)
prob = model.predict(review)

print("REVIEW: %s" % review)
print("\n")
print("La recesione è %s [%.6f]" % (prob_to_sentiment(prob), prob))